In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [75]:
import tensorflow as tf
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available 1


In [12]:
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.metrics import TruePositives, FalsePositives, TrueNegatives, FalseNegatives, BinaryAccuracy, Precision, Recall, AUC
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout, Activation, SpatialDropout2D, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import History, EarlyStopping, ReduceLROnPlateau
from typing import Tuple
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from src.preprocessing import train_val_test_split, parse_input_file

In [5]:
# directory to image folder - change this accordingly
DTIF = os.path.join('data', 'img')
DF_FILEPATH = 'list_category_img.txt'

In [68]:
df = parse_input_file(DF_FILEPATH, DTIF)
train_df, val_df, test_df = train_val_test_split(df)

In [77]:
train_datagen = ImageDataGenerator(
    rescale=1./255
)
val_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

In [ ]:
train_gen = train_datagen.flow_from_dataframe(
    train_df,
    weight_col=None, target_size=(75, 75), color_mode='rgb',
    batch_size=128,
    class_mode='categorical', 
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    weight_col=None, target_size=(75, 75), color_mode='rgb',
    batch_size=128,
    class_mode='categorical', 
    shuffle=False
)

Found 173513 validated image filenames belonging to 46 classes.


In [79]:
def base_model() -> Sequential:
    model = Sequential()
    model.add(Input((75, 75, 3)))    
    # Conv2D Layer
    model.add(Conv2D(filters=64, kernel_size=5, padding='same'))
    model.add(Activation('relu'))    
    model.add(SpatialDropout2D(0.5))

    # BatchNormalization
    model.add(BatchNormalization())
    model.add(Activation('relu'))    

    # Max Pooling
    model.add(MaxPooling2D(pool_size=(5, 5), strides = 5))

    model.add(Flatten())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.4))

    model.add(Dense(46, activation='softmax'))
    
    return model  

In [80]:
METRICS = [
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      AUC(name='prc', curve='PR')
]

model = base_model()
model.compile(optimizer='adam', loss=CategoricalCrossentropy(), metrics=METRICS)
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 75, 75, 64)        4864      
_________________________________________________________________
activation_14 (Activation)   (None, 75, 75, 64)        0         
_________________________________________________________________
spatial_dropout2d_7 (Spatial (None, 75, 75, 64)        0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 75, 75, 64)        256       
_________________________________________________________________
activation_15 (Activation)   (None, 75, 75, 64)        0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 14400)            

In [81]:
number_epochs = 3
early_stoppage = EarlyStopping(monitor='val_loss', patience=3)
variable_learning_rate = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=2, min_lr=0.0001)
history = model.fit(
    train_gen,
    epochs=number_epochs,
    validation_data=val_gen,
    callbacks=[variable_learning_rate, early_stoppage]    
)

Epoch 1/3
  17/5423 [..............................] - ETA: 36:52 - loss: 3.7997 - accuracy: 0.9781 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.6174 - prc: 0.0479

KeyboardInterrupt: 